<a href="https://colab.research.google.com/github/KingT5M/NARENDRA-CONCURRENT-FAULT/blob/main/NARENDRA_CONCURRENT_FAULT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import tensorflow as tf
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt 
import keras_tuner as kt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from scipy.stats import randint, uniform 
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Conv1D, BatchNormalization, MaxPooling1D, LSTM, Flatten, Dense, Dropout, Reshape
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, Callback
from kerastuner import HyperModel, Hyperband
from keras.layers import Input, Dense, GRU
from keras.models import Model


In [17]:
# File paths
file_paths = [
    'C:\\Users\\T5M\\Desktop\\CONCURRENT-FAULT\\DATASET\\DelayAPP.csv',
    'C:\\Users\\T5M\\Desktop\\CONCURRENT-FAULT\\DATASET\\GainRPM.csv',
    'C:\\Users\\T5M\\Desktop\\CONCURRENT-FAULT\\DATASET\\HealthyData.csv',
    'C:\\Users\\T5M\\Desktop\\CONCURRENT-FAULT\\DATASET\\NoiseAPP.csv',
    'C:\\Users\\T5M\\Desktop\\CONCURRENT-FAULT\\DATASET\\PacketLossAPP.csv'
]
# Load and preprocess the data
def load_and_preprocess_data(file_paths):
    data_frames = []
    for file_path in file_paths:
        if os.path.exists(file_path):
            data = pd.read_csv(file_path, header=None, usecols=[i for i in range(1, 10)])
            data_frames.append(data)
        else:
            print(f"File not found: {file_path}")
    if not data_frames:
        print("No data frames loaded.")
        return None
    concatenated_data = pd.concat(data_frames, ignore_index=True)
    print("Concatenated data shape:", concatenated_data.shape)
    concatenated_data = concatenated_data.sample(frac=1).reset_index(drop=True)
    print("Concatenated data shape after shuffling:", concatenated_data.shape)
    return concatenated_data

# Load and preprocess the data
concatenated_data = load_and_preprocess_data(file_paths)

Concatenated data shape: (42597, 9)
Concatenated data shape after shuffling: (42597, 9)


In [18]:
# Preprocess the concatenated data
def preprocess_data(concatenated_data):
    features = concatenated_data.iloc[:, :-1].values
    labels = concatenated_data.iloc[:, -1].values
    
    # Label Encoding
    label_encoder = LabelEncoder()
    labels_encoded = label_encoder.fit_transform(labels)
    
    # One-Hot Encoding
    onehot_encoder = OneHotEncoder()
    labels_encoded = labels_encoded.reshape(-1, 1)
    labels_onehot = onehot_encoder.fit_transform(labels_encoded).toarray()
    
    return features, labels_onehot

# Create sequences
def create_sequences(features, labels, sequence_length=30):
    X, y = [], []
    for i in range(len(features) - sequence_length + 1):
        end_ix = i + sequence_length
        seq_x, seq_y = features[i:end_ix], labels[end_ix - 1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Preprocess the data
features, labels_onehot = preprocess_data(concatenated_data)

# Split data into train, validation, and test sets
sequence_length = 30
X_seq, y_seq = create_sequences(features, labels_onehot, sequence_length)

X_train, X_temp, y_train, y_temp = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [19]:
# Denoising Autoencoder (DAE) for feature extraction
input_dim = X_train.shape[2]  # Corrected input_dim
latent_dim = 64

input_layer = Input(shape=(sequence_length, input_dim))
encoded = Dense(128, activation='relu')(input_layer)
encoded = Dense(latent_dim, activation='relu')(encoded)
decoded = Dense(128, activation='relu')(encoded)
decoded = Dense(input_dim, activation='sigmoid')(decoded)

dae = Model(input_layer, decoded)
dae.compile(optimizer='adam', loss='mean_squared_error')

dae.fit(X_train, X_train, epochs=50, batch_size=64, validation_split=0.2)

encoder = Model(input_layer, encoded)
features_dae_train = encoder.predict(X_train)
features_dae_val = encoder.predict(X_val)
features_dae_test = encoder.predict(X_test)

features_dae_train = np.array(features_dae_train)
features_dae_val = np.array(features_dae_val)
features_dae_test = np.array(features_dae_test)

# Gated Recurrent Unit (GRU) for feature learning
gru_input_shape = features_dae_train.shape[1:]

gru_model = Sequential()
gru_model.add(GRU(64, input_shape=gru_input_shape, return_sequences=True))
gru_model.add(GRU(64, return_sequences=True))
gru_model.add(GRU(64, return_sequences=True))
gru_model.add(GRU(64))

gru_model.add(Reshape((1, 64)))

gru_output_train = gru_model.predict(features_dae_train)
gru_output_val = gru_model.predict(features_dae_val)
gru_output_test = gru_model.predict(features_dae_test)

# Define metrics callback
class MetricsCallback(Callback):
    def __init__(self, validation_data):
        super(MetricsCallback, self).__init__()
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        X_val, y_val = self.validation_data
        y_pred = self.model.predict(X_val)
        y_pred_classes = tf.argmax(y_pred, axis=1)
        y_true = tf.argmax(y_val, axis=1)

        precision = precision_score(y_true, y_pred_classes, average='weighted')
        recall = recall_score(y_true, y_pred_classes, average='weighted')
        f1 = f1_score(y_true, y_pred_classes, average='weighted')

        print(f"Validation Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

        if precision >= 0.9886 and recall >= 0.9890 and f1 >= 0.9888:
            print("Achieved desired metrics. Stopping training.")
            self.model.stop_training = True


Epoch 1/50
426/426 [==============================] - 30s 34ms/step - loss: 875100480.0000 - val_loss: 874691712.0000
Epoch 2/50
426/426 [==============================] - 12s 28ms/step - loss: 875100800.0000 - val_loss: 874691712.0000
Epoch 3/50
426/426 [==============================] - 12s 27ms/step - loss: 875101248.0000 - val_loss: 874691712.0000
Epoch 4/50
426/426 [==============================] - 15s 36ms/step - loss: 875101120.0000 - val_loss: 874691712.0000
Epoch 5/50
426/426 [==============================] - 12s 29ms/step - loss: 875101056.0000 - val_loss: 874691712.0000
Epoch 6/50
426/426 [==============================] - 13s 31ms/step - loss: 875100672.0000 - val_loss: 874691712.0000
Epoch 7/50
426/426 [==============================] - 12s 29ms/step - loss: 875101184.0000 - val_loss: 874691712.0000
Epoch 8/50
426/426 [==============================] - 13s 30ms/step - loss: 875101056.0000 - val_loss: 874691712.0000
Epoch 9/50
426/426 [==============================] - 11

In [21]:
# Define fault_types
fault_types = ['delay-time', 'gain', 'healthy', 'noise', 'packetloss']

# Define hypermodel
class CustomHyperModel(HyperModel):
    def __init__(self, X_train, y_train, X_val, y_val, fault_types):
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
        self.fault_types = fault_types  # Store fault_types as an instance variable

    def build(self, hp):
        model = Sequential()

        # Hyperparameters
        cnn_layers = hp.Int('cnn_layers', min_value=0, max_value=5, default=5)
        lstm_layers = hp.Int('lstm_layers', min_value=0, max_value=5, default=4)
        dense_layers = hp.Int('dense_layers', min_value=0, max_value=5, default=0)
        epochs = hp.Int('epochs', min_value=50, max_value=900, default=850)
        max_pooling = hp.Int('max_pooling', min_value=0, max_value=1, default=1)
        dropout = hp.Int('dropout', min_value=0, max_value=2, default=0)
        batch_norm = hp.Int('batch_norm', min_value=0, max_value=2, default=2)
        batch_size = hp.Int('batch_size', min_value=64, max_value=150, default=64)
        learning_rate = hp.Float('learning_rate', min_value=0.0001, max_value=0.001, default=0.0005, sampling='linear')

        # CNN Layers
        for i in range(cnn_layers):
            model.add(Conv1D(filters=8, kernel_size=2, activation='relu', padding='same', input_shape=(sequence_length, latent_dim)))
            if batch_norm:
                model.add(BatchNormalization())

        # Max Pooling Layer
        if max_pooling:
            model.add(MaxPooling1D(pool_size=2))

        # LSTM Layers
        for i in range(lstm_layers):
            model.add(LSTM(units=64, activation='relu', return_sequences=True))
            if batch_norm:
                model.add(BatchNormalization())

        # Flatten Layer
        model.add(Flatten())

        # Dense Layers
        for i in range(dense_layers):
            model.add(Dense(units=64, activation='relu'))
            if dropout:
                model.add(Dropout(0.5))
            if batch_norm:
                model.add(BatchNormalization())

        # Output Layer
        model.add(Dense(units=len(fault_types), activation='softmax'))  # Adjusted to match the number of fault types

        # Compile the model
        model.compile(optimizer=Adam(learning_rate=learning_rate),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

        return model

hypermodel = CustomHyperModel(features_dae_train, y_train, features_dae_val, y_val, fault_types)



# Define the tuner
tuner = Hyperband(
    hypermodel,
    objective='val_accuracy',
    max_epochs=50,
    factor=3,
    directory='hyperparameters_tuning',
    project_name='fault_detection'
)

# Define metrics callback
metrics_callback = MetricsCallback((features_dae_val, y_val))

print(type(features_dae_train))
print(features_dae_train.shape)


# Perform the hyperparameter search
tuner.search(
    features_dae_train,
    y_train,
    validation_data=(features_dae_val, y_val),
    callbacks=[metrics_callback],
    epochs=50
)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best hyperparameters found: {best_hps}")

# Build the model with the best hyperparameters
best_model = tuner.hypermodel.build(best_hps)

# Train the model
best_model.fit(
    features_dae_train,
    y_train,
    validation_data=(features_dae_val, y_val),
    batch_size=best_hps.get('batch_size'),
    epochs=best_hps.get('epochs'),
    callbacks=[metrics_callback]
)

# Save the best model
best_model.save(r'C:\Users\T5M\Desktop\CONCURRENT-FAULT\GRAPHS & MODEL\best_model.h5')

# Calculate metrics on the test set
y_pred = best_model.predict(features_dae_test)
y_pred_classes = tf.argmax(y_pred, axis=1)
y_true = tf.argmax(y_test, axis=1)

precision = precision_score(y_true, y_pred_classes, average=None, zero_division=1)
recall = recall_score(y_true, y_pred_classes, average=None)
f1 = f1_score(y_true, y_pred_classes, average=None)

# Sensor fault types
fault_types = ['delay-time', 'gain', 'healthy', 'noise', 'packetloss']

# Plotting precision
plt.figure(figsize=(10, 5))
plt.bar(fault_types, precision)
plt.title('Precision')
plt.xlabel('Fault Types')
plt.ylabel('Precision (%)')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.savefig(r'C:\Users\T5M\Desktop\CONCURRENT-FAULT\GRAPHS & MODEL\precision.png')
plt.close()

# Plotting recall
plt.figure(figsize=(10, 5))
plt.bar(fault_types, recall)
plt.title('Recall')
plt.xlabel('Fault Types')
plt.ylabel('Recall (%)')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.savefig(r'C:\Users\T5M\Desktop\CONCURRENT-FAULT\GRAPHS & MODEL\recall.png')
plt.close()

# Plotting F1-score
plt.figure(figsize=(10, 5))
plt.bar(fault_types, f1)
plt.title('F1-score')
plt.xlabel('Fault Types')
plt.ylabel('F1-score (%)')
plt.xticks(rotation=45)
plt.grid(axis='y')
plt.tight_layout()
plt.savefig(r'C:\Users\T5M\Desktop\CONCURRENT-FAULT\GRAPHS & MODEL\f1_score.png')
plt.close()


Trial 6 Complete [00h 05m 48s]
val_accuracy: 0.2031947374343872

Best val_accuracy So Far: 0.2872915267944336
Total elapsed time: 04h 19m 23s



Search: Running Trial #7

Value             |Best Value So Far |Hyperparameter
2                 |5                 |cnn_layers
1                 |1                 |lstm_layers
3                 |3                 |dense_layers
593               |781               |epochs
1                 |0                 |max_pooling
2                 |0                 |dropout
2                 |2                 |batch_norm
131               |110               |batch_size
0.00093122        |0.000837          |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
3                 |3                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
  27/1065 [..............................] - ETA: 24s - loss: 2.1797 - accuracy: 0.2130

KeyboardInterrupt: 